In [1]:
import os, cv2
import tensorflow as tf
import matplotlib.pyplot as plt, numpy as np, pandas as pd
from tensorflow import keras

from centernet import CenterNet, decode

from data import VOCDataset
from losses import centernet_loss

In [2]:
input_shape = (512, 512)
backbone = 'resnet50'

epochs = 1
batch_size = 2
buffer_size = batch_size * 5
lr = 1e-2
freeze = True
finetune = True or freeze

data_path = "..\\..\\Centernet\\VOC2007"
train_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\train.txt"
val_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\val.txt"
test_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\test.txt"


train_dataset_raw = VOCDataset(data_path, input_shape, train_file,
                             batch_size, False)
train_dataset = train_dataset_raw.load_dataset()


val_dataset_raw = VOCDataset(data_path, input_shape, val_file,
                             batch_size, False)
val_dataset = val_dataset_raw.load_dataset()

########
test_dataset_raw = VOCDataset(data_path, input_shape, test_file,
                             batch_size, False)
test_dataset = test_dataset_raw.load_dataset()
########

vis_dataset_raw = VOCDataset(data_path, input_shape, train_file, 1, False)
vis_dataset = vis_dataset_raw.load_dataset().repeat()

steps_per_epoch = len(train_dataset_raw) // batch_size

loading the context of data_file ...: 550it [00:00, 550696.40it/s]
loading the context of data_file ...: 50it [00:00, ?it/s]
loading the context of data_file ...: 50it [00:00, 25043.61it/s]
loading the context of data_file ...: 550it [00:00, 552410.73it/s]


In [3]:
ckpt_path = './logs/test/20220608-144555/ckpt.h5'

model = CenterNet(train_dataset_raw.class_names,
                      backbone_weights='imagenet',
                      freeze=freeze,
                      finetune=finetune)

In [4]:
model.build(input_shape=(1, 512, 512, 3))
model.load_weights(ckpt_path)

In [5]:
hm_pred, wh_pred, reg_pred = model.predict(val_dataset)

In [6]:
detections_pred = decode(hm_pred, wh_pred, reg_pred, 1000)

In [7]:
def point_mse(arr, gt):
    pnt_x = int((arr[0] + arr[2])/2)
    pnt_y = int((arr[1] + arr[3])/2)

    xmin,ymin,xmax,ymax = gt

    gt_x = int((xmin + xmax)/2)
    gt_y = int((ymin + ymax)/2)

    return np.sqrt((gt_y-pnt_y)**2 + (gt_x-pnt_x)**2)

In [8]:
t = detections_pred

mse_result = []


for i in range(50):
    #topk_x1, topk_y1, topk_x2, topk_y2, scores, class_ids
    Glabella = []
    point_R3 = []
    Nasion = []


    for topk_x1, topk_y1, topk_x2, topk_y2, scores, class_ids in t[i]:

        if(class_ids==0):
            Glabella.append([topk_x1, topk_y1, topk_x2, topk_y2])

        elif(class_ids==1):
            point_R3.append([topk_x1, topk_y1, topk_x2, topk_y2])

        elif(class_ids==2):
            Nasion.append([topk_x1, topk_y1, topk_x2, topk_y2])

    
    #print(len(Glabella))
    
    Glabella = np.array(Glabella)
    point_R3 = np.array(point_R3)
    Nasion = np.array(Nasion)

    # ground truth
    Glabella_gt = val_dataset_raw[i][1][0]
    point_R3_gt = val_dataset_raw[i][1][1]
    Nasion_gt = val_dataset_raw[i][1][2]


    Glabella_mse = point_mse(Glabella[0], Glabella_gt)
    point_R3_mse = point_mse(point_R3[0], point_R3_gt)
    Nasion_mse = point_mse(Nasion[0], Nasion_gt)

    #print(Glabella_mse, point_R3_mse, Nasion_mse)


    mse_result.append([i+1, np.round(Glabella_mse, 4), np.round(point_R3_mse, 4), np.round(Nasion_mse, 4)])


    

In [18]:
import datetime

d = datetime.datetime.now()
today = f'{d.month}_{d.day}'

print(today)

# 이후 strftime() method 사용해볼 것!

6_21


In [19]:
mse_result_df = pd.DataFrame(mse_result, columns = ['Data_num', 'Glabella', 'R3', 'Nasion'], )

mse_result_df.to_csv(f'./csv/mse_{today}.csv', index=False)

In [20]:
mse_result_df

,Data_num,Glabella,R3,Nasion
0,1,5.8310,3.1623,10.1980
1,2,2.2361,5.0000,6.0000
2,3,6.3246,3.0000,7.2801
3,4,4.1231,3.6056,8.5440
4,5,6.4031,2.0000,8.2462
5,6,2.2361,4.1231,5.0000
6,7,0.0000,1.4142,7.0711
7,8,3.1623,2.2361,7.6158
8,9,3.1623,5.0000,12.2066
9,10,5.6569,3.1623,15.2971
